In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "init-gemini",
   "metadata": {},
   "outputs": [],
   "source": [
    "import google.generativeai as genai\n",
    "from google.colab import userdata\n",
    "\n",
    "# Učitavanje API ključa\n",
    "API_KEY = userdata.get('GEMINI_API_KEY')\n",
    "genai.configure(api_key=API_KEY)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "models-setup",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definicija modela\n",
    "model_fast = genai.GenerativeModel('gemini-2.0-flash-lite')\n",
    "model_advanced = genai.GenerativeModel('gemini-2.5-pro')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "prompt-optimizer",
   "metadata": {},
   "outputs": [],
   "source": [
    "import json, re\n",
    "\n",
    "def refine_prompt(original_prompt, result_text):\n",
    "    query = f'''\n",
    "    You act as a system for refining prompts. Analyze the original prompt and the produced output.\n",
    "    Return a JSON dictionary with keys:\n",
    "    - optimized_prompt\n",
    "    - feedback\n",
    "    - rationale\n",
    "\n",
    "    Original prompt: \"{original_prompt}\"\n",
    "    Model output: \"{result_text}\"\n",
    "    '''\n",
    "\n",
    "    resp = model_advanced.generate_content(query)\n",
    "    text = resp.text\n",
    "\n",
    "    match = re.search(r\"```json\\n([\\s\\S]*?)\\n```\", text)\n",
    "    if match:\n",
    "        parsed = json.loads(match.group(1))\n",
    "    else:\n",
    "        parsed = json.loads(text)\n",
    "\n",
    "    return parsed"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "loop",
   "metadata": {},
   "outputs": [],
   "source": [
    "def optimize_loop(start_prompt, rounds=5):\n",
    "    prompt_state = start_prompt\n",
    "\n",
    "    for step in range(1, rounds + 1):\n",
    "        print(f\"---- Iteracija {step} ----\")\n",
    "        print(\"Trenutni prompt:\", prompt_state)\n",
    "\n",
    "        quick_output = model_fast.generate_content(prompt_state).text\n",
    "        print(\"Brzi model (output):\", quick_output)\n",
    "\n",
    "        refined = refine_prompt(prompt_state, quick_output)\n",
    "\n",
    "        print(\"Feedback:\", refined.get('feedback'))\n",
    "        print(\"Rationale:\", refined.get('rationale'))\n",
    "        print(\"Novi prompt:\", refined.get('optimized_prompt'))\n",
    "        print()\n",
    "\n",
    "        prompt_state = refined.get('optimized_prompt')\n",
    "\n",
    "    return prompt_state"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "run",
   "metadata": {},
   "outputs": [],
   "source": [
    "start = \"Objasni u detalje pocetniku kako funkcionira motor auta.\"\n",
    "optimize_loop(start, 3)"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
